# 이번 주에는 총 3가지를 배운다.  

1. Variable
2. Funtion
3. Numeric diffential
  
--- 
개인적인 공부 
1. Python magic method
2. Python decorator

# Variable

이번 강의를 이어나갈 때, 모든 변수(데이터)를 저장하는 형태인듯

In [135]:
import numpy as np

In [136]:
def my_print(x):
    print(f"The type : {type(x)}")
    if isinstance(x,Variable):
        print(f"The data : {x.data}")
    else:
        print(f"The data : {x}")

In [137]:
class Variable(object):
    def __init__(self, x):
        self.data = x

In [138]:
x = Variable(np.array(3.0))

my_print(x)

The type : <class '__main__.Variable'>
The data : 3.0


# Function

가상화같은 느낌으로 이해하면 될 듯  
  
직접적으로 Function class를 사용하는 것이 아니라,  
모든 함수는 Function을 항상 상속하여, 구현되도록 함

In [139]:
class Function(object):
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        return output
    
    def forward(self, x):
        raise NotImplementedError() # raise와 assert차이 알아내기

In [140]:
#f = Function()
#output = f(x)

In [141]:
class Square(Function):
    def forward(self, x):
        return  x ** 2
    
class Exp(Function):
    def forward(self, x):
        return np.exp(x)

## Activation func

활성화 함수는 역전파 적용 안됨

1. Sigmoid = $\frac{1}{1+e^{-x}}$  
  
2. Tanh = $\frac{sinh^{x}}{cosh^{x}}$
  
3. Softmax = $\frac{e^{x_{i}}}{\sum^{N}_{j=1}e^{x_{j}}}$
  
4. ReLU = $Max(0, x)$ 

In [142]:
class Sigmoid(Function):
    def forward(self, x):
        return 1 / (1 + np.exp(-x))
    
class Tanh(Function):
    def forward(self, x):
        return np.sinh(x) / np.cosh(x)
    
class Softmax(Function):
    def forward(self, x):
        exps = np.exp(x)
        return exps / np.sum(exps)
    
class ReLU(Function):
    def forwrad(self, x):
        index = x > 0
        return x * index

In [143]:
x = Variable(np.array(10.))
f = Square()
y = f(x)

my_print(y)

The type : <class '__main__.Variable'>
The data : 100.0


In [144]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
my_print(y)

The type : <class '__main__.Variable'>
The data : 1.648721270700128


## Magic method & Decorator

call, init과 같은 유용한 magic method 및 데코레이터 조사

### Construction and Initialization
1. __new__
2. __init__
3. __del__

In [145]:
class Myclass(object):
    def __new__(cls, *args, **kwargs):
        # This method is used for ... I don't know well when we can use  and why we should use...
        # The point is this method is called before init method
        print("__new__ method called")
        inst = super().__new__(cls)
        return inst
    
    def __init__(self, name):
        print("__init__ method called")
        self.name = name

    def __del__(self):
        # This method is called, when you use "del ~~"
        print("__del__ method called")

In [146]:
myclass = Myclass("EunNam")

print(f"name: {myclass.name}")
del myclass

try:
    print(myclass.name)
except Exception as e:
    print(f"There is no myclass: {e}")

__new__ method called
__init__ method called
name: EunNam
__del__ method called
There is no myclass: name 'myclass' is not defined


### Representing

1. __repr__
2. __str__
3. __hash__

In [147]:
class Myclass(object):    
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        # It provides informations for re-generating class instance
        return f"Myclass('{self.name}')"

    def __str__(self):
        # print(instance)
        return f"Myclass named {self.name}"
    
    def __hash__(self):
        """
        The hash function is responsible for converting the input into a shorter, constant-length output. 
        At this time, it is basic to design the output to be different when the input is different. 
        It is mainly used to check the identity of two objects.
        """
        return hash((self.name))
    
    def __eq__(self, other):
        # obj1 == obj2 behavior
        return self.__hash__() == other.__hash__()

In [148]:
myclass1 = Myclass("EunNam")
myclass2 = Myclass("Cho")

print(myclass1)

if myclass1 == myclass2:
    print("Same")
else:
    print("differnt")

Myclass named EunNam
differnt


In [149]:
myclass1

Myclass('EunNam')

### Accessing Attribute

I think these methods is not used well.  
However, when we must keep class security, Must use these.  
It seems relevant to **_"Encapsulation"_**
  
#### **_So... Skip..._**
  
1. __getattr__
2. __getattribution__
3. __setattr__
4. __delattr__   

========================

1. __setitem__
2. __getitem__
3. __delitem__
4. __contains__

### Sequencing
 
1. __len__
2. __getitem__
3. __iter__
4. __next__

In [150]:
class MyContainer:
    def __init__(self, data):
        self.data = data
        self.index = 0

    def __iter__(self):
        # I don't know what this method excatly do.
        return self
    
    def __next__(self):
        # It makes this class iterate through For statement
        if self.index >= len(self.data):
            raise StopIteration
        result = self.data[self.index]
        self.index += 1
        return result

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]

In [151]:
container = MyContainer([1, 2, 3, 4, 5])

for item in container:
    print(item)

print(len(container)) 

print(container[2])  

1
2
3
4
5
5
3


## Decorator

함수나 클래스의 기능을 확장하거나 수정하는데 사용됨  
재사용률, 가독성, 간결성 향상가능  
  
하지만, Decorator를 이해하려면, **_클로저_** 를 알고 있어야 함

### Closure

In [4]:
# Example in func

def my_decorator(func):
    print(f"Something is happening before the function is called.")
    func()
    print(f"Something is happening after the function is called.")

def say_hello():
    print(f"Hello!")

my_decorator(say_hello)

Something is happening before the function is called.
Hello!
Something is happening after the function is called.


In [1]:
# Example in func

def my_decorator(func):
    def wrapper():
        print(f"Something is happening before the function is called.")
        func()
        print(f"Something is happening after the function is called.")
    return wrapper

@my_decorator
def say_hello():
    print(f"Hello!")

say_hello()

Something is happening before the function is called.
Hello!
Something is happening after the function is called.


In [2]:
def add_property(cls):
    class DecoratorClass(cls):
        @property
        def full_name(self):
            return f"{self.first_name} {self.last_name}"
    return DecoratorClass

@add_property
class Person:
    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name

person = Person("John", "Doe")

print(person.full_name)

John Doe


# Numeric diffential
  
역전파의 장점을 부각시키기 위해 미분을 간단히 구현  

컴퓨터로 $\lim\limits_{n\to\infty}$ 보낼 수 없으니, **_수치미분_** 으로 미분  
수치 미분은 $\lim\limits_{h\to0}\frac{f(x+h) - f(x)}{h}$ 대신, **_아주 작은 값_** $\epsilon$을 대입
  
아주 작은 값으로 **_근사_** 하는 것이라, 오차가 발생  
  
이를 **_전진차분_**(forward differentail) $\frac{f(x+\epsilon) - f(x)}{\epsilon}$ 이 아닌,   
**_중앙차분_**(centered differential) 으로 줄임 $\frac{f(x+\epsilon) - f(x-\epsilon)}{2\epsilon}$ 

In [152]:
def numerical_diff(f, x, eps=1e-4):
    # centered differential
    x1 = Variable(x.data - eps)
    x2 = Variable(x.data + eps)

    y1 = f(x1)
    y2 = f(x2)

    return (y2.data - y1.data) / (2 * eps)

In [153]:
f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)

my_print(dy)

The type : <class 'numpy.float64'>
The data : 4.000000000004


In [154]:
def f(x):
    A = Square()
    B = Exp()
    C = Square()
    return C(B(A(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f,x)

my_print(dy)

The type : <class 'numpy.float64'>
The data : 3.2974426293330694
